In [ ]:
import os
import sys
import gymnasium

: 

In [ ]:
import os
from PIL import Image

# Path to the directory with TIFF files
tiff_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/BF-C2DL-HSC/01"

# Get a sorted list of all TIFF files in the directory
tiff_files = sorted([f for f in os.listdir(tiff_dir) if f.endswith('.tif')])

# List to hold the image frames
frames = []

# Load all TIFF images
for tiff_file in tiff_files:
    file_path = os.path.join(tiff_dir, tiff_file)
    img = Image.open(file_path)
    frames.append(img)

# Save as GIF
output_gif_path = os.path.join(tiff_dir, "output.gif")
frames[0].save(output_gif_path, format='GIF', append_images=frames[1:], save_all=True, duration=200, loop=0)

print(f"GIF saved at {output_gif_path}")


In [ ]:
import os
from PIL import Image, ImageOps

# Path to the directory with TIFF files
tiff_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa/train/Fluo-N2DL-HeLa/01/"

# Get a sorted list of all TIFF files in the directory
tiff_files = sorted([f for f in os.listdir(tiff_dir) if f.endswith('.tif')])

# List to hold the normalized image frames
frames = []

# Normalize and load all TIFF images
for tiff_file in tiff_files:
    file_path = os.path.join(tiff_dir, tiff_file)
    img = Image.open(file_path)

    # Convert 16-bit image (I;16) to 8-bit (L)
    if img.mode == 'I;16':
        img = img.point(lambda i: i * (1 / 256)).convert('L')

    # Normalize the image to enhance visibility
    img = ImageOps.autocontrast(img)

    # Append the normalized image to the list
    frames.append(img)

# Save the frames as a GIF
output_gif_path = os.path.join("output_normalized.gif")
frames[0].save(output_gif_path, format='GIF', append_images=frames[1:], save_all=True, duration=200, loop=0)

print(f"Normalized GIF saved at {output_gif_path}")


In [ ]:
len(frames)

In [ ]:
import os
from PIL import Image

# Path to the directory with TIFF files
tiff_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa/train/Fluo-N2DL-HeLa/01_GT/TRA"

# Get a sorted list of all TIFF files in the directory
tiff_files = sorted([f for f in os.listdir(tiff_dir) if f.endswith('.tif')])

# List to hold the image frames
frames = []

# Load all TIFF images
for tiff_file in tiff_files:
    file_path = os.path.join(tiff_dir, tiff_file)
    img = Image.open(file_path)
    frames.append(img)

# Save as GIF
output_gif_path = os.path.join("Fluo_st_mask.gif")
frames[0].save(output_gif_path, format='GIF', append_images=frames[1:], save_all=True, duration=200, loop=0)
print(f"GIF saved at {output_gif_path}")

In [ ]:
tiff_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa/train/Fluo-N2DL-HeLa/01_GT/TRA"


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure, morphology
from PIL import Image

# Directory containing the TIFF files for segmentation masks
tiff_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa/train/Fluo-N2DL-HeLa/01_GT/TRA"

# Directory to save temporary frames
temp_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa_TRA/temp_frames"
os.makedirs(temp_dir, exist_ok=True)

# Function to load TIFF files from the directory
def load_tiff_files(tiff_dir):
    tiff_files = sorted([os.path.join(tiff_dir, f) for f in os.listdir(tiff_dir) if f.endswith('.tif')])
    return [io.imread(f) for f in tiff_files]

# Function to extract cell centroids from segmentation masks
def extract_cell_centroids(segmentation_mask):
    # Label connected components (cells)
    labels = measure.label(segmentation_mask, connectivity=2)
    properties = measure.regionprops(labels)
    
    # Extract centroids
    centroids = [prop.centroid for prop in properties]
    return centroids

# Function to plot cell centroids on each frame and save them as images
def save_frames_as_images(frames, temp_dir):
    for frame_index, frame in enumerate(frames):
        plt.figure(figsize=(10, 10))
        plt.imshow(frame, cmap='gray')
        
        # Extract centroids from the current frame
        centroids = extract_cell_centroids(frame)
        
        # Plot cell centroids
        for centroid in centroids:
            y, x = centroid
            plt.plot(x, y, 'ro', label=f'Cell {frame_index}')
        
        plt.title(f'Frame {frame_index}')
        plt.axis('off')
        plt.savefig(os.path.join(temp_dir, f'frame_{frame_index:03d}.png'))
        plt.close()

# Function to create a GIF from saved images
def create_gif_from_images(temp_dir, gif_filename):
    images = []
    for file_name in sorted(os.listdir(temp_dir)):
        if file_name.endswith('.png'):
            file_path = os.path.join(temp_dir, file_name)
            images.append(Image.open(file_path))
    images[0].save(gif_filename, save_all=True, append_images=images[1:], optimize=False, duration=200, loop=0)

# Main function to process and create GIF
def main():
    frames = load_tiff_files(tiff_dir)  # Load TIFF images (segmentation masks)
    save_frames_as_images(frames, temp_dir)  # Save frames with cell centroids as images
    create_gif_from_images(temp_dir, 'cell_paths.gif')  # Create GIF from images

if __name__ == "__main__":
    main()


In [ ]:
import os
import numpy as np
from skimage import io, measure
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio


# Directory containing the TIFF files for segmentation masks
tiff_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa/train/Fluo-N2DL-HeLa/01_GT/TRA"

# Directory to save temporary frames
temp_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa_TRA/temp_frames"
os.makedirs(temp_dir, exist_ok=True)

# Function to load TIFF files from the directory
def load_tiff_files(tiff_dir):
    tiff_files = sorted([os.path.join(tiff_dir, f) for f in os.listdir(tiff_dir) if f.endswith('.tif')])
    return [io.imread(f) for f in tiff_files]
# New function to track cells across frames
def track_cells_across_frames(all_centroids):
    tracked_cells = {}
    for i, centroids in enumerate(all_centroids):
        if i == 0:
            for label, centroid in centroids.items():
                tracked_cells[label] = [(centroid[0], centroid[1], i)]
        else:
            prev_centroids = all_centroids[i-1]
            for label, centroid in centroids.items():
                min_distance = float('inf')
                closest_label = None
                for prev_label, prev_centroid in prev_centroids.items():
                    distance = np.linalg.norm(np.array(centroid) - np.array(prev_centroid))
                    if distance < min_distance:
                        min_distance = distance
                        closest_label = prev_label
                if closest_label in tracked_cells:
                    tracked_cells[closest_label].append((centroid[0], centroid[1], i))
                else:
                    tracked_cells[label] = [(centroid[0], centroid[1], i)]
    return tracked_cells

# New function to visualize trajectories in 3D
def visualize_trajectories(tracked_cells, output_dir):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    for cell_id, trajectory in tracked_cells.items():
        xs, ys, zs = zip(*trajectory)
        ax.plot(xs, ys, zs, label=f'Cell {cell_id}')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Frame')
    plt.title('Cell Trajectories')
    plt.legend()
    
    plt.savefig(os.path.join(output_dir, 'trajectories_3d.png'))
    plt.close(fig)

    # Create animated GIF
    for i in range(len(tracked_cells[list(tracked_cells.keys())[0]])):
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        
        for cell_id, trajectory in tracked_cells.items():
            xs, ys, zs = zip(*trajectory[:i+1])
            ax.plot(xs, ys, zs, label=f'Cell {cell_id}')
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Frame')
        plt.title(f'Cell Trajectories at Frame {i}')
        plt.legend()
        
        plt.savefig(os.path.join(output_dir, f'trajectory_{i}.png'))
        plt.close(fig)

    # Create GIF
    images = []
    for i in range(len(tracked_cells[list(tracked_cells.keys())[0]])):
        images.append(imageio.imread(os.path.join(output_dir, f'trajectory_{i}.png')))
    
    imageio.mimsave(os.path.join(output_dir, 'cell_trajectories.gif'), images, duration=0.5)

# Modified main function
def main():
    frames = load_tiff_files(tiff_dir)
    centroids_list = track_cells(frames)
    tracked_cells = track_cells_across_frames(centroids_list)
    save_centroids_to_file(centroids_list, output_file)
    
    # Visualize trajectories
    visualize_trajectories(tracked_cells, os.path.dirname(output_file))

if __name__ == "__main__":
    main()


In [ ]:
create_gif_from_images(temp_dir, 'cell_paths.gif')  # Create GIF from images

In [ ]:
# %pip install requests

In [ ]:
import os
import requests
from zipfile import ZipFile

# List of dataset URLs
dataset_urls = [
    "http://data.celltrackingchallenge.net/training-datasets/BF-C2DL-HSC.zip",
    "http://data.celltrackingchallenge.net/training-datasets/BF-C2DL-MuSC.zip",
    "http://data.celltrackingchallenge.net/training-datasets/DIC-C2DH-HeLa.zip",
    "http://data.celltrackingchallenge.net/training-datasets/Fluo-C2DL-Huh7.zip",
    "http://data.celltrackingchallenge.net/training-datasets/Fluo-N2DH-GOWT1.zip",
    "http://data.celltrackingchallenge.net/training-datasets/Fluo-N2DL-HeLa.zip",
    "http://data.celltrackingchallenge.net/training-datasets/PhC-C2DH-U373.zip",
    "http://data.celltrackingchallenge.net/training-datasets/Fluo-N2DH-SIM+.zip"
]

# Directory to save the datasets
save_dir = "/home/komal.kumar/Documents/Cell/datasets/data/CTC/Training"
os.makedirs(save_dir, exist_ok=True)

# Function to download and extract a zip file
def download_and_extract(url, save_dir):
    local_filename = os.path.join(save_dir, url.split('/')[-1])
    print(f"Downloading {url}...")
    
    # Download the file
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    
    # Extract the file
    print(f"Extracting {local_filename}...")
    with ZipFile(local_filename, 'r') as zip_ref:
        zip_ref.extractall(save_dir)
    
    # Remove the zip file after extraction
    os.remove(local_filename)
    print(f"Downloaded and extracted: {url}")

# Download and extract all datasets
for url in dataset_urls:
    download_and_extract(url, save_dir)

print("All datasets downloaded and extracted.")


import os
import requests
from zipfile import ZipFile

# List of dataset URLs (modified to point to test datasets)
test_dataset_urls = [
    "http://data.celltrackingchallenge.net/test-datasets/BF-C2DL-HSC.zip",
    "http://data.celltrackingchallenge.net/test-datasets/BF-C2DL-MuSC.zip",
    "http://data.celltrackingchallenge.net/test-datasets/DIC-C2DH-HeLa.zip",
    "http://data.celltrackingchallenge.net/test-datasets/Fluo-C2DL-Huh7.zip",
    "http://data.celltrackingchallenge.net/test-datasets/Fluo-N2DH-GOWT1.zip",
    "http://data.celltrackingchallenge.net/test-datasets/Fluo-N2DL-HeLa.zip",
    "http://data.celltrackingchallenge.net/test-datasets/PhC-C2DH-U373.zip",
    "http://data.celltrackingchallenge.net/test-datasets/Fluo-N2DH-SIM+.zip"
]

# Directory to save the test datasets
save_dir = "/home/komal.kumar/Documents/Cell/datasets/data/CTC/Test"
os.makedirs(save_dir, exist_ok=True)

# Function to download and extract a zip file
def download_and_extract(url, save_dir):
    local_filename = os.path.join(save_dir, url.split('/')[-1])
    print(f"Downloading {url}...")
    
    # Download the file
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    
    # Extract the file
    print(f"Extracting {local_filename}...")
    with ZipFile(local_filename, 'r') as zip_ref:
        zip_ref.extractall(save_dir)
    
    # Remove the zip file after extraction
    os.remove(local_filename)
    print(f"Downloaded and extracted: {url}")

# Download and extract all test datasets
for url in test_dataset_urls:
    download_and_extract(url, save_dir)

print("All test datasets downloaded and extracted.")



In [ ]:
import os
import requests
from zipfile import ZipFile

# List of dataset URLs (modified to point to test datasets)
test_dataset_urls = [
    "http://data.celltrackingchallenge.net/test-datasets/BF-C2DL-HSC.zip",
    "http://data.celltrackingchallenge.net/test-datasets/BF-C2DL-MuSC.zip",
    "http://data.celltrackingchallenge.net/test-datasets/DIC-C2DH-HeLa.zip",
    "http://data.celltrackingchallenge.net/test-datasets/Fluo-C2DL-Huh7.zip",
    "http://data.celltrackingchallenge.net/test-datasets/Fluo-N2DH-GOWT1.zip",
    "http://data.celltrackingchallenge.net/test-datasets/Fluo-N2DL-HeLa.zip",
    "http://data.celltrackingchallenge.net/test-datasets/PhC-C2DH-U373.zip",
    "http://data.celltrackingchallenge.net/test-datasets/Fluo-N2DH-SIM+.zip"
]

# Directory to save the test datasets
save_dir = "/home/komal.kumar/Documents/Cell/datasets/data/CTC/Test"
os.makedirs(save_dir, exist_ok=True)

# Function to download and extract a zip file
def download_and_extract(url, save_dir):
    local_filename = os.path.join(save_dir, url.split('/')[-1])
    print(f"Downloading {url}...")
    
    # Download the file
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    
    # Extract the file
    print(f"Extracting {local_filename}...")
    with ZipFile(local_filename, 'r') as zip_ref:
        zip_ref.extractall(save_dir)
    
    # Remove the zip file after extraction
    os.remove(local_filename)
    print(f"Downloaded and extracted: {url}")

# Download and extract all test datasets
for url in test_dataset_urls:
    download_and_extract(url, save_dir)

print("All test datasets downloaded and extracted.")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skimage import io, measure
import imageio

# Directory containing the TIFF files for segmentation masks
tiff_dir = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa/train/Fluo-N2DL-HeLa/01_GT/TRA"

# Function to load TIFF files from the directory
def load_tiff_files(tiff_dir):
    tiff_files = sorted([os.path.join(tiff_dir, f) for f in os.listdir(tiff_dir) if f.endswith('.tif')])
    return [io.imread(f) for f in tiff_files]

# Function to extract cell centroids from segmentation masks
def extract_cell_centroids(segmentation_mask):
    labels = measure.label(segmentation_mask, connectivity=2)
    properties = measure.regionprops(labels)
    centroids = {prop.label: prop.centroid for prop in properties}
    return centroids

# Function to track cell centroids across frames
def track_cells(frames):
    all_centroids = []
    for frame in frames:
        centroids = extract_cell_centroids(frame)
        all_centroids.append(centroids)
    
    return all_centroids

# Function to visualize cell trajectories and save as GIF
def visualize_trajectories_3d_gif(frames, centroids_list, gif_path='trajectories.gif'):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    colors = plt.cm.jet(np.linspace(0, 1, len(centroids_list[0])))  # Color map for cells
    
    # Dictionary to store cell trajectories
    trajectories = {}
    images = []  # To store the frames for the GIF

    for frame_index, (frame, centroids) in enumerate(zip(frames, centroids_list)):
        for cell_id, centroid in centroids.items():
            x, y = centroid
            z = frame_index  # Time step as the Z-axis

            # Initialize trajectory for new cells
            if cell_id not in trajectories:
                trajectories[cell_id] = ([], [], [])

            # Update trajectory for each cell
            trajectories[cell_id][0].append(x)
            trajectories[cell_id][1].append(y)
            trajectories[cell_id][2].append(z)

        # Clear plot and redraw
        ax.clear()
        ax.set_xlim([0, frame.shape[1]])
        ax.set_ylim([0, frame.shape[0]])
        ax.set_zlim([0, len(frames)])

        # Plot the trajectories of all cells
        for cell_id, (x_traj, y_traj, z_traj) in trajectories.items():
            ax.plot(x_traj, y_traj, z_traj, color=colors[cell_id % len(colors)], marker='o')

        ax.set_xlabel('X axis')
        ax.set_ylabel('Y axis')
        ax.set_zlabel('Time (Z axis)')
        ax.set_title(f"Frame {frame_index + 1}")

        # Save current figure to buffer (for GIF)
        fig.canvas.draw()
        image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        images.append(image_from_plot)

    # Save all frames as a GIF
    imageio.mimsave(gif_path, images, fps=5)  # Adjust FPS as needed

    plt.show()

# Main function to process and visualize centroid data
def main():
    frames = load_tiff_files(tiff_dir)  # Load TIFF images (segmentation masks)
    centroids_list = track_cells(frames)  # Track cells across frames
    visualize_trajectories_3d_gif(frames, centroids_list, gif_path='3d_trajectories.gif')  # Visualize trajectories in 3D and save as GIF

if __name__ == "__main__":
    main()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
path = "/home/komal.kumar/Documents/Cell/datasets/celltrack/2D/Fluo-N2DL-HeLa/train/Fluo-N2DL-HeLa/01_GT/TRA/man_track000.tif"
img = plt.imread(path)

In [ ]:
plt.imshow(img)

In [ ]:
img.shape